In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings


gemini_chat_model = ChatGoogleGenerativeAI(
    model="gemini-pro",
    api_key=os.environ["GOOGLE_API_KEY"],
    temperature=0
    )

gemini_embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")



I0000 00:00:1723200700.358827 6055895 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723200700.362188 6055895 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723200700.364689 6055895 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_pinecone import PineconeVectorStore

default_vector_store = PineconeVectorStore.from_existing_index(
    index_name=os.environ['PINECONE_DEFAULT_INDEX'],
    embedding= gemini_embedding_model,
)

default_retriever=default_vector_store.as_retriever()

In [17]:
template = """Act as a chatbot and nswer the question based only on the following context:
{context}
Question: {question}
"""


In [18]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [19]:
prompt = ChatPromptTemplate.from_template(template)

In [20]:
chain = (
    RunnableParallel({"context": default_retriever, "question": RunnablePassthrough()})
    | prompt
    | gemini_chat_model
    | StrOutputParser()
)

In [21]:
chain.invoke("Can I get a quotation?")

'The best way to get a quote is by going to the order now page and following the steps. You will get the quote instantly before you place the order. Or if you have a special request, you can simply email us at sales@customlanyard.in and we will be more than glad to help you out.'